## Exploratory Analysis

That's it for some preliminary cleaning. Don't worry, there will be more. Let's start to look in a bit more detail at the data, though. In this section, we're going to start to write some code that's typical for day-to-day data cleaning tasks.

In [1]:
from load_data import dta

In [11]:
import pandas as pd

We can use `info` to get some high-level information about the data.

In [3]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 1965287 to 413268
Data columns (total 15 columns):
address            25000 non-null object
aka_name           24709 non-null object
city               24981 non-null object
dba_name           25000 non-null object
facility_type      24787 non-null category
inspection_date    25000 non-null datetime64[ns]
inspection_type    25000 non-null category
latitude           24865 non-null float64
license_           24996 non-null float64
longitude          24865 non-null float64
results            25000 non-null category
risk               24995 non-null category
state              24994 non-null object
violations         23908 non-null object
zip                24990 non-null object
dtypes: category(4), datetime64[ns](1), float64(3), object(7)
memory usage: 2.4+ MB


And `describe` goes into a bit more detail for the *numeric* types, of which we don't have many here.

In [4]:
dta.describe()

,latitude,license_,longitude
count,24865.000000,2.499600e+04,24865.000000
mean,41.879734,1.475028e+06,-87.675890
std,0.081968,8.912679e+05,0.058372
min,41.644670,0.000000e+00,-87.914428
25%,41.828059,1.000632e+06,-87.707209
50%,41.891528,1.884173e+06,-87.666377
75%,41.939792,2.134724e+06,-87.634533
max,42.021064,8.700606e+06,-87.525125


We could do the same for the categorical types.

In [6]:
dta.select_dtypes(['category']).head()

,facility_type,inspection_type,results,risk
inspection_id,,,,
1965287,Restaurant,Canvass,Pass w/ Conditions,Risk 1 (High)
1329698,Restaurant,Canvass,Pass,Risk 1 (High)
470787,Restaurant,SFP,Fail,Risk 1 (High)
68091,Restaurant,Canvass,Pass,Risk 2 (Medium)
1335320,Mobile Food Dispenser,License Re-Inspection,Fail,Risk 3 (Low)


In [7]:
dta.select_dtypes(['category']).describe()

,facility_type,inspection_type,results,risk
count,24787,25000,25000,24995
unique,226,40,7,4
top,Restaurant,Canvass,Pass,Risk 1 (High)
freq,16954,12487,15915,18249


### GroupBy

In [15]:
dta.risk.value_counts()

Risk 1 (High)      18249
Risk 2 (Medium)     5069
Risk 3 (Low)        1675
All                    2
Name: risk, dtype: int64

Now, let's ask the most obvious question. Which are the best and the worst restaurants? We'll want to use pandas `GroupBy` functionality to implement the `split-apply-combine` pattern.

The idea here is that we **split** the data by some key or set of keys then **apply** a function to each group and then **combine** the outputs back into a single DataFrame.

In [9]:
dta.longitude.head()

inspection_id
1965287   -87.757220
1329698   -87.775907
470787    -87.668429
68091     -87.645155
1335320   -87.687507
Name: longitude, dtype: float64

First, let's see how many result categories there are. We can use `value_counts` to answer this question. 

In [12]:
with pd.option_context("max.rows", 10):
    print(dta.results.value_counts())

Pass                    15915
Fail                     5760
Pass w/ Conditions       2931
Out of Business           246
No Entry                  107
Not Ready                  40
Business Not Located        1
Name: results, dtype: int64


In [13]:
dta.head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
68091,2804 N CLARK ST,Wells Street Popcorn,CHICAGO,Wells Street Popcorn,Restaurant,2010-02-01,Canvass,41.932921,1954774.0,-87.645155,Pass,Risk 2 (Medium),IL,NaN,60657
1335320,2409 N WESTERN AVE,CHICAGO CUPCAKE,CHICAGO,CHICAGO CUPCAKE LLC.,Mobile Food Dispenser,2013-05-03,License Re-Inspection,41.925218,2232391.0,-87.687507,Fail,Risk 3 (Low),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60647


Ok, let's group on the inspection `results` column and see who the best and worst are.

When we call the `groupby` method we get back a `DataFrameGroupBy` object.

In [16]:
grouper = dta.groupby(dta.results)

In [20]:
grouper.head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_type,latitude,license_,longitude,results,risk,state,violations,zip
inspection_id,,,,,,,,,,,,,,,
1965287,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26,Canvass,41.880237,1991820.0,-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1329698,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06,Canvass,41.931250,2099479.0,-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
470787,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03,SFP,41.979884,1933748.0,-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
68091,2804 N CLARK ST,Wells Street Popcorn,CHICAGO,Wells Street Popcorn,Restaurant,2010-02-01,Canvass,41.932921,1954774.0,-87.645155,Pass,Risk 2 (Medium),IL,NaN,60657
1335320,2409 N WESTERN AVE,CHICAGO CUPCAKE,CHICAGO,CHICAGO CUPCAKE LLC.,Mobile Food Dispenser,2013-05-03,License Re-Inspection,41.925218,2232391.0,-87.687507,Fail,Risk 3 (Low),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60647
1228169,3481 S DR MARTIN LUTHER KING JR DR,DUNKIN DONUTS/BASKIN ROBBINS,CHICAGO,DUNKIN DONUTS/BASKIN ROBBINS,Restaurant,2013-09-16,Canvass,41.831186,1823777.0,-87.616821,Pass,Risk 2 (Medium),IL,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,60616
1285582,3201-3203 W ARMITAGE AVE,STREET SIDE CAFE,CHICAGO,STREET SIDE CAFE,Restaurant,2012-09-27,Canvass,41.917291,84648.0,-87.707052,Pass w/ Conditions,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60647
557486,5215 W CHICAGO AVE BLDG E2,"GOLDEN HOOK FISH, CHICKEN",CHICAGO,"GOLDEN HOOK FISH, CHICKEN",Restaurant,2011-08-11,Canvass,41.894815,2069859.0,-87.756302,Pass,Risk 2 (Medium),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60651
74468,4445 S Drexel (900E),KING COLLEGE PREP HIGH SCHOOL,CHICAGO,KING SELECTIVE ENROL.H/S,School,2010-05-26,Canvass,41.813695,46371.0,-87.603838,Pass,Risk 1 (High),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60653


You can access the variables on this object, the same as a DataFrame, and any code called will execute within the groups.

In [18]:
result = grouper.dba_name.value_counts()

The result is a Series with a `MultiIndex`.

In [19]:
result

results               dba_name                              
Business Not Located  CITGO SUPER WASH & GAS                     1
Fail                  SUBWAY                                    42
                      DUNKIN DONUTS                             33
                      MCDONALD'S                                20
                      HAROLD'S CHICKEN SHACK                    14
                      CHIPOTLE MEXICAN GRILL                    11
                      DUNKIN DONUTS / BASKIN ROBBINS            10
                      7-ELEVEN                                   9
                      MCDONALDS                                  9
                      POTBELLY SANDWICH WORKS                    9
                      POTBELLY SANDWICH WORKS LLC                9
                      JIMMY JOHNS                                8
                      LAS ISLAS MARIAS                           8
                      KFC                                        7
 

In [21]:
result.index.names

FrozenList(['results', 'dba_name'])

We can index on the first element in a `MultiIndex` using square brackets and then use `sort_values` to find those restaurants that had a result of Fail the most.

In [22]:
with pd.option_context('max.rows', 15):
    print(result["Fail"].sort_values(ascending=False))

dba_name
SUBWAY                            42
DUNKIN DONUTS                     33
MCDONALD'S                        20
HAROLD'S CHICKEN SHACK            14
CHIPOTLE MEXICAN GRILL            11
DUNKIN DONUTS / BASKIN ROBBINS    10
MCDONALDS                          9
                                  ..
MERCY'S TOBACCO                    1
MERCY HOME FOR BOYS AND GIRLS      1
MERCER 113                         1
MELLOS PEANUT COMPANY              1
MEHRAB SUPER MARKET                1
MEERATH KABAB                      1
HALSTED FOOD                       1
Name: dba_name, Length: 4424, dtype: int64


Take a closer look above. Looks like we have some more data cleaning to do.

In [23]:
with pd.option_context('max.rows', 15):
    print(result["Pass"].sort_values(ascending=False))

dba_name
SUBWAY                            219
DUNKIN DONUTS                     118
MCDONALD'S                         42
7-ELEVEN                           38
AU BON PAIN                        29
POTBELLY SANDWICH WORKS LLC        29
CHIPOTLE MEXICAN GRILL             27
                                 ... 
KATERINA'S STREET OF DREAMS         1
KATAKANA & FIESTA & KOKO GRILL      1
KASEY'S TAVERN                      1
KARYN'S ON GREEN                    1
KARLA'S KITCHEN INC.                1
KARLA'S KITCHEN                     1
CITY DOUGH                          1
Name: dba_name, Length: 9328, dtype: int64


This is probably not the right way to think about this since there are many more Subways than local establishments.

We could instead look at the ratio of Fail to Pass, though, of course, this isn't perfect either. 

Sometimes, it's not *always* obvious how to go about computing things that you want to compute. The `get_group` method allows you to pull out one of the split DataFrames and try your apply function on it.

## Exercise

GroupBy the `dba_name`. Use `get_group` to pull out the "MCDONALD'S" group. Write a function that calculates the relative number of Fail to Pass for this group. Run this function on the McDonald's group.

In [ ]:
# Type your solution here

In [ ]:
# %load solutions/get_group.py
grouper = dta.groupby(dta.dba_name)
mcd = grouper.get_group("MCDONALD'S")


def relative_results(df):
    values = df.value_counts()
    return values['Fail'] / values['Pass']


relative_results(mcd.results)

# And we see McDonald's failed 50% as many inspections as it Passed.


We can run this on everything, but it's going to be a little slow. Let's look at another way to approach this problem.

Here group by *both* the inspection results and the DBA name. Then we ask for the `size` of each one of these groups.

In [ ]:
result = dta.groupby((dta.results, dta.dba_name)).size()

In [ ]:
result

We can use the `div` method to divide these for us. As you can see the indices don't line up, but we don't have to worry about it. Pandas take care of index alignment for us.

In [ ]:
result["Fail"]

In [ ]:
result["Pass"]

In [ ]:
ratio = result["Fail"].div(result["Pass"])
ratio.sort_values(ascending=False, inplace=True)
ratio

We have a lot of `NaN`s in the results from division-by-zero. We can drop those with a call to `dropna`. Also note that pandas lets you decide whether to treat `inf` as an NA.

In [ ]:
with pd.option_context("use_inf_as_null", True,
                       "max.rows", 15):
    print(ratio.dropna())

We might still not be wholly satisfied with our rules around comparisons here. First, we're looking at restaurant names not particular establishments. What does the distribution of inspection visits for establishments look like.

In [ ]:
with pd.option_context("max.rows", 10):
    grouper = dta.groupby((dta.address, dta.dba_name))
    print(grouper.size().describe())

Ok, let's make things a little more challenging. Let's see what the Fail:Pass ratio is for restaurants with at least 3 visits that involved a high risk level. 


Now we're starting to get into some much more powerful pandas constructs.

In [ ]:
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .rename('n_visits')
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))

visited

Let's unpack this. The first thing to note is how this code is organized. Each one of these methods return a pandas data structure on which we call the next method. This is called **method chaining**. We use the same trick seen above to split strings across lines to split several method calls by including the code between `()`.

Next, we see several new methods. The first is **query**. When subsetting a DataFrame we have a few options. As we save above, we can index a DataFrame using integers. Likewise, we could pass an object of booleans as well.

In [ ]:
dta.risk == "Risk 1 (High)"

In [ ]:
dta.loc[dta.risk == "Risk 1 (High)"]

Always using indexing can be verbose, however. You may need compound statements, for example.

In [ ]:
dta.loc[(dta.risk == "Risk 1 (High)") | (dta.risk == "Risk 1 (Medium)")].head()

Instead, by using query we could write the following, which is slightly easier.

In [ ]:
dta.query("(risk == 'Risk 1 (High)') | (risk == 'Risk 1 (Medium)')").head()

## Exercise

Use `query` to find the visits that are to restaurants and that are complaint re-inspections.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/query.py

```python
visited = (dta.query("risk == 'Risk 1 (High)'")
           .groupby(('address', 'dba_name'))
           .size()
           .rename('n_visits')  # size returns a nameless series
           .reset_index()  # make into a DataFrame
           .query("n_visits >= 4"))
```


The next new method is the **rename** method. We use this to rename the unnamed Series returned by `size`.

Finally, we filter on restaurants with 4 or more total visits.

The final piece is computing the Fail:Pass ratio of these restaurants. To do this, we need to take the output we've created `visited` and line our original data up with these addresses and DBA names.

In [ ]:
visited.head()

We can do this by using the **merge** method. Merge allows us to make two pandas DataFrames into a single DataFrame. By default, the `merge` method will join together two DataFrames on common columns, using an inner join method (a set intersection).

In [ ]:
merged_visits = visited.merge(dta)
merged_visits.head()

Now, we need to redo the analysis we started above for McDonald's. We take these merged DataFrames, group them by the inspection results, the address, and the DBA name and ask for the size of each group.

In [ ]:
(merged_visits
 .groupby(('results', 'address', 'dba_name'))
 .size()).head()

## Exercise

Take this result and **pipe** it (using the `pipe` method) to a function that computes Fail/Pass. Make sure your result does not have any missing values, and sort it such that those with the highest Fail/Pass ratios are highest.

The **pipe** method allows for including user-defined functions in method chains. It takes the output of the thing on the left, and passes it to the thing on the right.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/pipe_results.py

## Exercise

Now, take everything that we've done above, from `dta` to this final result, and put it together into a single method chain.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/complete_chain.py

Often, I'll try to place a method chain to get the data ready for more exploratory work at the top of a notebook, so I can proceed with any analyses.

Now, let's go back to our original data and add in the unstacked violations to the information that's unchanging. Recall from the previous notebook that we unstack the violations as follows. There are two new things to note here though. We add in a `to_frame` method to turn the unstacked Series into a DataFrame, and we `rename` the unnamed column in the resulting DataFrame back to `violations`.

In [ ]:
(dta.violations
 .str.split("|", expand=True)
 .unstack()
 .dropna()
 .reset_index(level=0, drop=True)
 .str.strip()
 .rename('violations')
 .to_frame())

First, we need to drop the violations from the original DataFrame, then we need to merge it with the unstacked violations Series that we created before. You can use `drop` to do this.

In [ ]:
dta.drop(["violations"], axis='columns').head()

## Exercise

Drop the original violations from `dta`, and **join** this to the unstacked violations as computed above. You'll probably want to use a **right join**.

We use **join** here rather than **merge**. Join uses merge under the hood but conveniently allows us to join on the indices of the two DataFrames by default. One other difference is that join uses an inner merge by default, but that's not what we want here. Since we drop the null violations on the right-hand side DataFrame, we want to do a right join. 

In [ ]:
# Type your solution here

In [ ]:
%load solutions/join_violations.py

Now that we have a relatively clean DataFrame, let's ask a few more questions. 

First, how many unique violations do we have?

In [ ]:
dta.shape

In [ ]:
dta.violations.head()

In [ ]:
dta.violations.unique().shape

Is this true? Do we really think there are this many violation numbers? Probably not. We can use the `str` accessor and some more munging to answer this. Here we pass a **regular expression** to `str.extract`. Extract expects a *capture group*, indicated by `()`. The regular expression `(\d+\)(?=\.)` means capture 1 or more (`+`) digits (`\d`) that is followed by (`(?=)`) a period `\.`. We escape the period because a plain `.` is a wildcard for any character.

In [ ]:
(dta.violations
 .str.extract("(\d+)(?=\.)", expand=False)
 .astype(int))

So how many unique violations do we have?

In [ ]:
np.sort(
    dta.violations
        .str.extract("(\d+)(?=\.)", expand=False)
        .astype(int)
        .unique()
).shape

Second, can we figure out how many times an establishment previously failed an inspection (within the sample we have)? How might we approach this? 

First, we want to restrict the data to just a single row for each inspection. Since we merged everything with the unstacked violations above, we'll need to use `drop_duplicates` to do this.

In [ ]:
visits = dta.drop_duplicates(["address", "dba_name", "inspection_date"])

We're going to rely on some pandas time-series functionality to do this, so we will need to ensure that the inspection dates are sorted within each group. GroupBy will preserve this.

In [ ]:
visits = visits.sort_values(["address", "dba_name", "inspection_date"])

In [ ]:
grouper = visits.groupby((visits.address, visits.dba_name))

Ok, we might ask, "now what?" Remember the trick to pull out groups? Let's use it to work with something we can think about.

In [ ]:
group_key = list(grouper.groups.keys())[0]

In [ ]:
group_key

In [ ]:
group = grouper.get_group(group_key)

In [ ]:
group

In [ ]:
group[['inspection_date', 'results']]

Since, we need this to be backwards looking, we will **shift** the data by one visit. Shifting will move the data around by either a number of periods or a frequency. In this case, we use a number of periods and shift forward by 1 period.

In [ ]:
group.shift(1)[['inspection_date', 'results']]

If we take the cumulative sum of this, we'll have an accurate picture of previous failures.

In [ ]:
(group.shift(1).results == 'Fail').cumsum()

In [ ]:
visit_num = grouper.apply(lambda df: (df.shift(1).results == 'Fail').cumsum())

In [ ]:
visit_num.head(n=15)

In [ ]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
))

In [ ]:
dta.head()

In [ ]:
visit_num.head()

In [ ]:
visit_num.reset_index(level=[0, 1], drop=True).head()

In [ ]:
(visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
))

In [ ]:
dta.join((visit_num.reset_index(level=[0, 1], drop=True)
 .to_frame()
 .rename(
    columns={'results': 'num_fails'}
)))